# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import re

import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=nubank user=postgres password=nubank")
cur = conn.cursor()

In [3]:
for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

for query in create_table_queries:
        cur.execute(query)
        conn.commit()

In [4]:
#VERIFY THE DB TABLES
cur.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
print(cur.fetchall())

[('accounts_table',), ('country_table',), ('city_table',), ('costumers_table',), ('d_month_table',), ('d_time_table',), ('d_week_table',), ('d_year_table',), ('pix_movements_table',), ('state_table',), ('transfer_ins_table',), ('transfer_outs_table',)]


In [5]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.csv'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process table data
In this first part, you'll perform ETL on the tables

In [6]:
table_files = get_files("tables/tables")

In [7]:
filepath = table_files[1]

In [8]:
df = pd.read_csv(filepath)
df.head()
df_1 = df[['city_id','city','state_id']]

In [9]:
df_1.head()

,city_id,city,state_id
0,670134511382200832,São José do Goiabal,2755422274446512640
1,1734898293086970880,Pederneiras,2066774635771587840
2,1215291016113604608,Unistalda,1255006025180785920
3,1751285020827958784,Córrego do Bom Jesus,2755422274446512640
4,2840661058008040960,Nova Monte Verde,1169007872647055360


In [10]:
for i, row in df_1.iterrows():
        cur.execute(city_table_insert, list(row))
conn.commit()
conn.close()

In [8]:
for file in filepath:
    print(file)
    f = open(file, 'r')
    print(os.path.basename(file).replace('.csv',''))


c:\Users\Filipe\Documents\Nubank-Analytics-Engineer-Case\tables\tables\accounts_table.csv
accounts_table
c:\Users\Filipe\Documents\Nubank-Analytics-Engineer-Case\tables\tables\city_table.csv
city_table
c:\Users\Filipe\Documents\Nubank-Analytics-Engineer-Case\tables\tables\costumers_table.csv
costumers_table
c:\Users\Filipe\Documents\Nubank-Analytics-Engineer-Case\tables\tables\country_table.csv
country_table
c:\Users\Filipe\Documents\Nubank-Analytics-Engineer-Case\tables\tables\d_month_table.csv
d_month_table
c:\Users\Filipe\Documents\Nubank-Analytics-Engineer-Case\tables\tables\d_time_table.csv
d_time_table
c:\Users\Filipe\Documents\Nubank-Analytics-Engineer-Case\tables\tables\d_weekday_table.csv
d_weekday_table
c:\Users\Filipe\Documents\Nubank-Analytics-Engineer-Case\tables\tables\d_week_table.csv
d_week_table
c:\Users\Filipe\Documents\Nubank-Analytics-Engineer-Case\tables\tables\d_year_table.csv
d_year_table
c:\Users\Filipe\Documents\Nubank-Analytics-Engineer-Case\tables\tables\pix_

In [9]:



for file in filepath:
    print(file)
    f = open(file, 'r')
    x = os.path.basename(file).replace('.csv','')

    copy_sql = """
           COPY %s FROM stdin WITH CSV HEADER
           DELIMITER as ','
           """
    tmp_df = "./tables/tables/tmp.csv"
    df.to_csv(tmp_df,index=False , header=False)

    try:
        
        cur.copy_from(f, x, sep=",", columns=)
        conn.commit()
    except psycopg2.Error as e: 
        print("Error: Inserting Rows")
        print (e)

c:\Users\Filipe\Documents\Nubank-Analytics-Engineer-Case\tables\tables\accounts_table.csv
c:\Users\Filipe\Documents\Nubank-Analytics-Engineer-Case\tables\tables\city_table.csv
Error: Inserting Rows
invalid input syntax for type integer: "SÃ£o JosÃ© do Goiabal"
CONTEXT:  COPY city_table, line 2, column city_id: "SÃ£o JosÃ© do Goiabal"

c:\Users\Filipe\Documents\Nubank-Analytics-Engineer-Case\tables\tables\costumers_table.csv
Error: Inserting Rows
current transaction is aborted, commands ignored until end of transaction block

c:\Users\Filipe\Documents\Nubank-Analytics-Engineer-Case\tables\tables\country_table.csv
Error: Inserting Rows
current transaction is aborted, commands ignored until end of transaction block

c:\Users\Filipe\Documents\Nubank-Analytics-Engineer-Case\tables\tables\d_month_table.csv
Error: Inserting Rows
current transaction is aborted, commands ignored until end of transaction block

c:\Users\Filipe\Documents\Nubank-Analytics-Engineer-Case\tables\tables\d_time_table.c

In [ ]:
for file in filepath:
    df = pd.read_csv(file)
    df.head()
    print(df.isna().sum())
    

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [ ]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']]
song_data.head()

In [ ]:
# Save the dataframe to disk
tmp_df = "./tmp_dataframe.csv"
song_data.to_csv(tmp_df,index=False , header=False)
f = open(tmp_df, 'r')
    
#start = time.time()
cur.copy_from(f, 'song_table', sep=",")
#end = time.time()

In [ ]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0].tolist()
song_data

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [ ]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [ ]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0].tolist()
artist_data

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [ ]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [ ]:
log_files = get_files("data/log_data")

In [ ]:
filepath_log = log_files
filepath_log

In [ ]:
df_log = pd.read_json(filepath_log[0], lines=True)
for file1 in range(1,len(filepath_log)):
    df_log1 = pd.read_json(filepath_log[file1], lines=True)
    df_log = df_log.append(df_log1, ignore_index = True)
df = df_log
df_log.info()

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
df = df.query('page == "NextSong"')
df.info()

In [ ]:
t = df['date'] = pd.to_datetime(df['ts'],unit='ms')

In [ ]:
time_data = [t, df['date'].dt.hour, df['date'].dt.day, df['date'].dt.weekofyear, df['date'].dt.month, df['date'].dt.year, df['date'].dt.dayofweek]
column_labels = ['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday'] 

In [ ]:
time_df = pd.DataFrame.from_dict(dict(zip(column_labels, time_data)))
time_df.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df["userId"] = pd.to_numeric(user_df["userId"])
user_df = user_df.drop_duplicates(subset=['userId'])
user_df.head(5)

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.